#### Certifique-se que instalou o chromedriver na pasta e rode a célula abaixo

In [1]:
#!pip install selenium

#Chromedriver
#https://googlechromelabs.github.io/chrome-for-testing/#stable

import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

path_chromedriver='C:/projetos/webscrappers/chromedriver.exe'
driver = webdriver.Chrome(service=Service(path_chromedriver))
driver.get('https://www2.tjal.jus.br/cpopg/open.do')

pagina_consulta = driver.current_window_handle

def reset():
    paginas = driver.window_handles
    for pagina in paginas:
        if pagina != pagina_consulta:
            driver.switch_to.window(pagina)
            driver.close()

    driver.switch_to.window(pagina_consulta)

#### Após abrir o navegador, rode a célula abaixo

In [2]:
def ctrlClick(element):
    ActionChains(driver) \
        .key_down(Keys.CONTROL) \
        .click(element) \
        .key_up(Keys.CONTROL) \
        .perform()
    
def getTeacherByName(name):

    select_tipo_busca = Select(driver.find_element(by=By.NAME, value='cbPesquisa'))
    filtro_nome = driver.find_element(by=By.ID, value='campo_NMPARTE')

    select_tipo_busca.select_by_value('NMPARTE')
    time.sleep(0.5)

    filtro_nome.clear()
    filtro_nome.send_keys(name)
    time.sleep(0.5)

    botao_consultar = driver.find_element(by=By.ID, value='botaoConsultarProcessos')
    ctrlClick(botao_consultar)
    time.sleep(2.0)
    driver.switch_to.window(driver.window_handles[1])

    if len(driver.find_elements(By.CSS_SELECTOR, '.alert + #mensagemRetorno')) == 1:
        return False

    # Apenas 1 resultado
    if len(driver.find_elements(By.CSS_SELECTOR, '.linkPasta.btn.btn-secondary.btn-space#linkPasta')) == 1:
        botao_pasta = driver.find_element(by=By.ID, value='linkPasta')
        ctrlClick(botao_pasta)
        time.sleep(2.0)
        driver.switch_to.window(driver.window_handles[2])
        return True
    
    # Apenas 2+ resultados
    listagemProcessos = driver.find_element(By.ID, 'listagemDeProcessos').find_elements(By.TAG_NAME, 'li')
    if len(listagemProcessos) > 1:
        linkPrimeiroProcesso = listagemProcessos[0].find_elements(By.CLASS_NAME, 'linkProcesso')[0]
        ctrlClick(linkPrimeiroProcesso)
        time.sleep(2.0)
        driver.switch_to.window(driver.window_handles[2])
        botao_pasta = driver.find_element(by=By.ID, value='linkPasta')
        ctrlClick(botao_pasta)
        time.sleep(2.0)
        driver.switch_to.window(driver.window_handles[3])
        return True

    # Nenhum resultado
    return False

def createRegex(name, cpf):
    firstName = name.split(' ')[0]
    cpf = cpf.replace('#',r'\d').replace('.',r'\.')
    return fr'{firstName}(?P<antes>[\w\W]*)(?P<cpf>{cpf})(?P<depois>[\w\W]*)\n'

def getCPFfromQualification(name, cpfAnonimo):
    import re

    driver.find_element(by=By.ID, value='primeiraPaginaButton').click()
    time.sleep(2.0)

    document_frame = driver.find_element(by=By.ID, value='documento')
    driver.switch_to.frame(document_frame)
    time.sleep(2.0)

    text = driver.find_element(by=By.ID, value='viewer').text
    pattern = re.compile(createRegex(name, cpfAnonimo))

    match = pattern.search(text)

    if match:
        return match.group('cpf')
    else:
        return 'Não encontrado'


#### Faça login e acesse a tela de consulta de processos

In [ ]:
# 2@&DixCep$

#### TESTES

In [ ]:
# TESTE: 1 resultado
name = 'ITALEQUELMA LIMA DE MELO'
cpfAnonimo = '###.005.3##-##'
getTeacherByName(name)
cpf = getCPFfromQualification(name, cpfAnonimo)
print(cpf)

In [ ]:
# TESTE: 2+ resultados
name = 'MARIA JEANE CORREIA CURVELO CAVALCANTI'
cpfAnonimo = '###.008.2##-##'
getTeacherByName(name)
cpf = getCPFfromQualification(name, cpfAnonimo)
print(cpf)

In [ ]:
# TESTE: 0 resultados
name = 'ERIVANIA GOMES CARVALHO'
getTeacherByName(name)

In [ ]:
# RESET
reset()

#### INICIO DO LOOP

In [ ]:
script_url = 'https://script.google.com/macros/s/AKfycbzyDcg-KuHK0CksMjJU6F_klpjDSBX3fdg-AFxu3hNPBgaMRGvRd8h0o4NrvU7DayxJ/exec?projeto=consultaTJAL'

import requests
import json

def getNextFromBacklog():
    response = requests.get(script_url)

    if response.status_code == 200:
        return response.json()
    else:
        print('Error:', response.status_code)

def postResultToBacklog(dados):
    response = requests.post(script_url, 
                         data=json.dumps(dados), 
                         headers={'Content-Type': 'application/json'})
    if response.status_code != 200:
        print('Error:', response.status_code)

while True:
    try:
        professor = getNextFromBacklog()
        name = professor['nome']
        cpfAnonimo = professor['cpfAnonimo']
        if getTeacherByName(name):
            cpf = getCPFfromQualification(name, cpfAnonimo)
            reset()
            postResultToBacklog({
                    'linha': professor['linha'], 
                    'cpf': cpf, 
                    'status': 'sucesso'
                }
            )
        else:
            reset()
            postResultToBacklog({'linha': professor['linha'], 'cpf': 'Não encontrado', 'status': 'erro'})

    except Exception as e:
        reset()
        postResultToBacklog({'linha': professor['linha'], 'cpf': 'erro', 'status': 'erro'})

In [37]:
def searchByNumber(numero):
    numero_limpo = numero.replace('.','').replace('-','')
    filtro_numero = driver.find_element(By.ID, "numeroDigitoAnoUnificado")
    filtro_numero.clear()
    filtro_numero.send_keys(numero_limpo[:13])
    
    filtro_comarca = driver.find_element(By.ID, "foroNumeroUnificado")
    filtro_comarca.clear()
    filtro_comarca.send_keys(numero_limpo[-4:])
    time.sleep(0.5)

    botao_consultar = driver.find_element(by=By.ID, value='botaoConsultarProcessos')
    ctrlClick(botao_consultar)

def get_cover_data():
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)
    driver.find_element(by=By.CSS_SELECTOR, value=".unj-link-collapse").click()
    time.sleep(1)
    
    numero = driver.find_element(by=By.CSS_SELECTOR, value="#numeroProcesso").text
    elements = driver.find_elements(by=By.CSS_SELECTOR, value=".unj-tag")
    tags = "".join([e.text for e in elements])
    classe = driver.find_element(by=By.CSS_SELECTOR, value="#classeProcesso").text
    assunto = driver.find_element(by=By.CSS_SELECTOR, value="#assuntoProcesso").text
    foro = driver.find_element(by=By.CSS_SELECTOR, value="#foroProcesso").text
    vara = driver.find_element(by=By.CSS_SELECTOR, value="#varaProcesso").text
    juiz = driver.find_element(by=By.CSS_SELECTOR, value="#juizProcesso").text
    distribuicao = driver.find_element(by=By.CSS_SELECTOR, value="#dataHoraDistribuicaoProcesso").text
    valor = driver.find_element(by=By.CSS_SELECTOR, value="#valorAcaoProcesso").text
    partes = driver.find_elements(by=By.CSS_SELECTOR, value=".nomeParteEAdvogado")
    ativo = partes[0].text.replace('\n', '| ').replace("  ", "").lstrip()
    passivo = partes[1].text.replace('\n', '| ').replace("  ", "").lstrip()
    return [numero, tags, classe, assunto, foro, vara, juiz, distribuicao, valor, ativo, passivo ]

reset()

In [39]:
project_path = 'C:/projetos/webscrappers/tjal'
import csv

def save(array, output_file, method):
    with open(f'{project_path}/{output_file}', method, encoding='UTF-8', newline='') as f:
        writer = csv.writer(f, delimiter=';')
        for row in array:
            writer.writerow(row)

def get_list(todo_file, done_file):
    todo = []
    with open(f'{project_path}/{todo_file}', 'r', encoding='UTF-8') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            todo.append(row[0])

    done = []
    with open(f'{project_path}/{done_file}', 'r', encoding='UTF-8') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            done.append(row[0])

    todo_list = set(todo) - set(done)
    
    return list(todo_list)



In [42]:
todo = get_list("todo.csv", "done.csv")
for numero in todo:
    try:
        #numero = "0707348-90.2018.8.02.0058"
        searchByNumber(numero.lstrip())
        valores = get_cover_data()
        print(valores)
        save([valores], 'done.csv', 'a')
        reset()
    except:
        reset()